# Extração e Classificação de BI-RADS em Laudos de Mamografia

## Introdução Técnica

Este notebook implementa um sistema automatizado para extração, classificação e processamento de informações de BI-RADS (Breast Imaging Reporting and Data System) a partir de laudos de mamografia do Grupo Fleury. O sistema é projetado para identificar pacientes com exames em atraso com base na classificação BI-RADS e programar lembretes para acompanhamento médico adequado.

### Objetivo Principal

O objetivo principal deste notebook é extrair automaticamente as classificações BI-RADS (0-6) de laudos de mamografia, calcular datas de retorno recomendadas com base nessa classificação, e identificar pacientes elegíveis para notificações de acompanhamento. Pacientes com BI-RADS 1, 2 e 3 são incluídos em uma lista de ativação para monitoramento e notificações de retorno.

### Tecnologias Utilizadas

- **Framework de Processamento**: Apache Spark (PySpark)
- **Armazenamento de Dados**: Delta Lake
- **Manipulação de Dados**: PySpark SQL e DataFrame API
- **Processamento de Texto**: Expressões Regulares (RegEx)
- **Monitoramento e Alertas**: Octoops (Sentinel)
- **Logging**: Biblioteca padrão Python (logging)

### Fluxo de Trabalho/Etapas Principais

1. **Configuração do Ambiente**: Instalação de dependências e configuração de logging
2. **Definição de Parâmetros**: Configuração das tabelas e filtros para consultas SQL
3. **Extração de BI-RADS**: Consulta SQL complexa que:
   - Utiliza RegEx para extrair classificações BI-RADS dos laudos
   - Transforma códigos romanos em valores numéricos (I→1, II→2, etc.)
   - Calcula valores mínimo, máximo e atual de BI-RADS
4. **Processamento de Datas de Retorno**: Cálculo das datas previstas de retorno com base na classificação BI-RADS
5. **Filtragem e Transformação**: Aplicação de transformações nos dados e filtragem de pacientes elegíveis
6. **Persistência de Dados**: Salvamento dos dados processados em tabelas Delta

### Dados Envolvidos

- **Fonte de Dados**: 
  - Tabela: `refined.saude_preventiva.fleury_laudos`
  - Filtro principal: `linha_cuidado = 'mama'`
  - Tipos de exame: 'MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D'

- **Tabelas de Destino**:
  - `refined.saude_preventiva.fleury_laudos_mama_birads`: Dados gerais dos laudos com classificação BI-RADS
  - `refined.saude_preventiva.fleury_laudos_mama_birads_ativacao`: Subconjunto com pacientes elegíveis para notificações

- **Colunas Principais**:
  - `ficha`, `id_item`, `id_subitem`: Identificadores do exame
  - `MIN_BIRADS`, `MAX_BIRADS`, `BIRADS`: Valores de classificação BI-RADS
  - `dth_pedido`: Data do pedido do exame
  - `dth_previsao_retorno`: Data calculada para retorno do paciente
  - `retorno_cliente`: Tempo recomendado para retorno em meses (12 para BI-RADS 1-2, 6 para BI-RADS 3)

### Resultados/Saídas Esperadas

1. **Tabela Estruturada de BI-RADS**: Tabela Delta contendo todos os exames de mamografia com classificação BI-RADS extraída
2. **Lista de Ativação**: Tabela Delta contendo apenas pacientes:
   - Com BI-RADS 1, 2 ou 3
   - Sem retorno registrado
   - Do sexo feminino entre 40-75 anos
   - Pertencentes a marcas específicas do grupo

### Pré-requisitos

- **Ambiente Databricks**: Com acesso ao Delta Lake e capacidade de execução de PySpark
- **Dependência Principal**: Pacote `octoops` para monitoramento e alertas
- **Acesso a Dados**: Permissões de leitura na tabela `refined.saude_preventiva.fleury_laudos`
- **Permissões de Escrita**: Para criar ou atualizar as tabelas de destino

### Considerações Importantes

- **Extração via RegEx**: A extração de BI-RADS usa expressões regulares sofisticadas para identificar padrões como "BIRADS X", "CATEGORIA X" ou notações em algarismos romanos
- **Recomendações de Retorno**: Seguem o protocolo médico padrão:
  - BI-RADS 1-2: Retorno em 12 meses (360 dias)
  - BI-RADS 3: Retorno em 6 meses (180 dias)
  - BI-RADS 4-6: Sem recomendação automática de retorno (requer avaliação médica)
- **Deduplicação**: A tabela de ativação remove duplicatas por `ficha` para evitar múltiplas notificações para o mesmo paciente
- **Tratamento de Erros**: Sistema robusto com captura de exceções e notificações via Sentinel em caso de falhas

## Instalação de Dependências: Octoops

Esta célula realiza a instalação da biblioteca `octoops` via pip. Esta biblioteca é essencial para o funcionamento do notebook, pois:

- Fornece funcionalidades de monitoramento e alerta para pipelines de dados
- Permite enviar notificações em caso de falhas no processamento
- Integra-se com sistemas de monitoramento para rastreamento de execuções

O comando `%pip install octoops` utiliza a sintaxe "magic command" do Jupyter para instalar a biblioteca diretamente no ambiente atual de execução, sem necessidade de reiniciar o kernel manualmente.

In [ ]:
%pip install octoops

## Verificação da Instalação do Octoops

Esta célula executa o comando `pip show octoops` para verificar se a biblioteca foi instalada corretamente e exibir informações detalhadas sobre a versão instalada. 

A saída esperada incluirá:
- Nome do pacote
- Versão
- Sumário (descrição breve)
- Autor
- Localização de instalação
- Dependências

Esta verificação é importante para confirmar que a biblioteca está disponível no ambiente antes de prosseguir com o restante do código que depende dela.

In [ ]:
pip show octoops

## Reinicialização do Python

Esta célula reinicia o interpretador Python usando a função `dbutils.library.restartPython()`, específica do ambiente Databricks. 

Esta operação é necessária após a instalação de novas bibliotecas para garantir que:
- As novas dependências sejam carregadas corretamente no ambiente
- As alterações de versões de bibliotecas sejam aplicadas
- Quaisquer configurações de ambiente modificadas pela instalação sejam ativadas

Após a reinicialização, o kernel Python será reiniciado, mantendo as células já executadas, mas com as novas bibliotecas disponíveis para uso nas células subsequentes.

In [ ]:
dbutils.library.restartPython()

## Importação de Bibliotecas

Esta célula importa todas as bibliotecas e módulos necessários para o funcionamento do notebook. As importações podem ser categorizadas da seguinte forma:

### PySpark SQL e DataFrame
- `col`, `year`, `month`, `dayofmonth`, `when`, `lit`, `expr`, `to_timestamp`: Funções para manipulação de colunas em DataFrames
- `DateType`: Tipo de dados para datas
- `DataFrame`: Classe para tipagem
- `datediff`, `to_date`: Funções para operações com datas

### Bibliotecas Padrão Python
- `datetime`: Para manipulação de datas e horários
- `logging`: Para registro de logs
- `sys`: Para interação com o sistema
- `traceback`: Para captura detalhada de erros

### Octoops (Monitoramento)
- `OctoOps`: Biblioteca principal para monitoramento
- `Sentinel`: Componente específico para envio de alertas

Estas importações fornecem as ferramentas necessárias para processar dados com PySpark, manipular datas, gerenciar erros e enviar alertas durante a execução do pipeline.

In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel

## Configuração do Logger

Esta célula inicializa um objeto logger para registrar eventos durante a execução do notebook. O logger é configurado usando:

- `logging.getLogger(__name__)`: Cria um logger associado ao nome do módulo atual

A configuração do logger permite:
- Registro padronizado de mensagens informativas, avisos e erros
- Rastreamento centralizado das etapas de execução
- Integração com o sistema de logging do Databricks

O logger será utilizado posteriormente no código para registrar eventos importantes, como avisos sobre DataFrames vazios ou informações sobre operações de salvamento de dados.

In [ ]:
logger = logging.getLogger(__name__)

## Configuração das Tabelas e Filtros

Esta célula define parâmetros cruciais para a execução do pipeline, incluindo nomes de tabelas de destino e filtros SQL.

### Variáveis/Tabelas Definidas

- **`table_birads`**: Nome da tabela principal de destino para armazenar todos os laudos processados com BI-RADS
- **`table_birads_ativacao`**: Nome da tabela de ativação para pacientes elegíveis para notificações
- **`where_clause`**: Cláusula SQL dinâmica para processamento incremental

### Lógica Detalhada

1. **Definição de Tabelas**: Define os caminhos completos para as tabelas Delta
2. **Processamento Incremental**:
   - Verifica se a tabela principal já existe usando `spark.catalog.tableExists()`
   - Se existir, configura a cláusula WHERE para processar apenas registros mais recentes que o último processamento
   - Isso otimiza o processamento ao evitar reprocessar registros antigos

3. **Definição do Filtro de Ativação**:
   - Configura um filtro SQL complexo para selecionar pacientes elegíveis para notificações
   - Critérios incluem:
     - Pacientes sem retorno registrado (`eleg.ficha IS NULL`)
     - Com BI-RADS 1, 2 ou 3 (`brd.BIRADS IN (1, 2, 3)`)
     - Realizaram exames específicos (`sigla_exame IN ('MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D')`)
     - Sexo feminino (`UPPER(flr.sexo_cliente) = 'F'`)
     - Faixa etária apropriada (`idade_cliente >= 40 AND idade_cliente < 76`)
     - Pertencentes a marcas específicas do grupo Fleury

Estes parâmetros determinam quais registros serão processados e como serão filtrados para as tabelas de destino.

In [ ]:
table_birads = "refined.saude_preventiva.fleury_laudos_mama_birads"
table_birads_ativacao = "refined.saude_preventiva.fleury_laudos_mama_birads_ativacao"
where_clause = ""
 
if spark.catalog.tableExists(table_birads):
    where_clause = f"""
    WHERE
        flr._datestamp >= (
            SELECT MAX(brd._datestamp)
            FROM {table_birads} brd
        )
    """
 
filtro_ativacao = """
    WHERE
        eleg.ficha IS NULL
        AND brd.BIRADS IN (1, 2, 3)
        AND flr.sigla_exame IN ('MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D')
        AND UPPER(flr.sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        )
        AND UPPER(flr.marca) ILIKE ANY(
            'AMAIS - BA',
            'AMAIS - PE',
            'AMAIS - SP',
            'Felippe Mattoso',
            'FLEURY',
            'IR',
            'LABS AMAIS'
        )
"""

## Consulta SQL para Extração de BI-RADS

Esta célula implementa uma consulta SQL complexa para extrair classificações BI-RADS de laudos de mamografia e criar dois DataFrames principais: um com todos os laudos processados e outro apenas com pacientes elegíveis para ativação.

### Estrutura da Consulta

A consulta utiliza Common Table Expressions (CTEs) para organizar o processamento em etapas lógicas:

1. **CTE `base`**: 
   - Extrai os identificadores básicos dos exames (`ficha`, `id_item`, `id_subitem`)
   - Usa expressões regulares sofisticadas para extrair menções a BI-RADS dos laudos
   - Transforma algarismos romanos (I, II, III, etc.) em valores numéricos
   - Filtra laudos da linha de cuidado 'mama' e tipos específicos de exame

2. **CTE `dados_birads`**: 
   - Calcula métricas agregadas de BI-RADS:
     - `MIN_BIRADS`: Menor valor de BI-RADS encontrado no laudo
     - `MAX_BIRADS`: Maior valor de BI-RADS encontrado no laudo
     - `BIRADS`: Valor mais recente/final (último elemento do array)

3. **CTE `dados_laudos`**: 
   - Coleta dados demográficos e administrativos
   - Calcula a idade do cliente 
   - Aplica o filtro incremental (`where_clause`)

4. **Consulta Final**: 
   - Combina os dados das CTEs através de JOINs
   - Une com a tabela de retornos elegíveis (`fleury_retorno_elegivel_ficha`)
   - Aplica o filtro de ativação quando apropriado

### DataFrames Gerados

- **`df_spk`**: DataFrame principal contendo todos os laudos com classificações BI-RADS extraídas
- **`df_spk_ativacao`**: DataFrame filtrado contendo apenas pacientes elegíveis para ativação (notificações)

### Aspectos Técnicos Importantes

- **Processamento de Texto**: Uso extensivo de funções RegEx para limpar e extrair BI-RADS
- **Transformação de Tipos**: Conversão entre tipos de dados (string para inteiro, romanos para arábicos)
- **Funções Analíticas**: Uso de `ARRAY_MIN`, `ARRAY_MAX` e `TRY_ELEMENT_AT` para análise de arrays
- **Formatação Dinâmica**: A consulta usa `.format()` para inserir dinamicamente as cláusulas SQL geradas anteriormente

In [ ]:
query = """
WITH base AS (
    SELECT
        flr.ficha,
        flr.id_item,
        flr.id_subitem,
        REGEXP_EXTRACT_ALL(
            REGEXP_EXTRACT(
                REGEXP_REPLACE(UPPER(flr.laudo_tratado), r'[-:®]|\xa0', ''),
                r'(?mi)(AVALIA[CÇ][AÃ]O|CONCLUS[AÃ]O|IMPRESS[AÃ]O|OPINI[AÃ]O)?(.*)', 2
            ),
            r"(?mi)(BIRADS|CATEGORIA|CAT\W)\s*(\d+\w*|VI|V|IV|III|II|I)\W*\w?(BIRADS|CATEGORIA)?(\W|$)", 2
        ) AS RAW_BIRADS,
        FILTER(
            TRANSFORM(RAW_BIRADS, x ->
                CASE
                    WHEN x = "I" THEN 1
                    WHEN x = "II" THEN 2
                    WHEN x = "III" THEN 3
                    WHEN x = "IV" THEN 4
                    WHEN x = "V" THEN 5
                    WHEN x = "VI" THEN 6
                    WHEN TRY_CAST(x AS INT) > 6 THEN NULL
                    ELSE REGEXP_REPLACE(x, r'[^0-9]', '')
                END
            ), x -> x IS NOT NULL
        ) AS CAT_BIRADS
    FROM refined.saude_preventiva.fleury_laudos flr
    WHERE
        flr.linha_cuidado = 'mama'
        AND flr.sigla_exame IN ('MAMOG', 'MAMOGDIG', 'MAMOPROT', 'MAMOG3D')
),
 
dados_birads AS (
    SELECT
        *,
        ARRAY_MIN(CAT_BIRADS) AS MIN_BIRADS,
        ARRAY_MAX(CAT_BIRADS) AS MAX_BIRADS,
        TRY_ELEMENT_AT(CAST(CAT_BIRADS AS ARRAY<INT>), -1) AS BIRADS
    FROM base
),
 
dados_laudos AS (
    SELECT
        flr.linha_cuidado,
        flr.id_unidade,
        flr.id_ficha,
        flr.id_item,
        flr.id_subitem,
        flr.ficha,
        flr.id_exame,
        flr.id_cliente,
        flr.pefi_cliente,
        flr.sigla_exame,
        flr.id_marca,
        flr.marca,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        flr.sexo_cliente,
        flr.dth_pedido,
        flr._datestamp
    FROM refined.saude_preventiva.fleury_laudos flr
    {where_clause}
)
 
SELECT
    flr.* except(idade_cliente),
    brd.MIN_BIRADS,
    brd.MAX_BIRADS,
    brd.BIRADS,
 
    eleg.dth_pedido        AS dth_pedido_retorno_elegivel,
    eleg.ficha             AS ficha_retorno_elegivel,
    eleg.siglas_ficha      AS siglas_ficha_retorno_elegivel,
    eleg.marca             AS marca_retorno_elegivel,
    eleg.unidade           AS unidade_retorno_elegivel,
    eleg.convenio          AS convenio_retorno_elegivel,
    eleg.valores_exame     AS valores_exame_retorno_elegivel,
    eleg.valores_ficha     AS valores_ficha_retorno_elegivel,
    eleg.qtd_exame         AS qtd_exame_retorno_elegivel,
    eleg.secao             AS secao_retorno_elegivel,
    eleg.dias_entre_ficha  AS dias_entre_ficha_elegivel
FROM dados_laudos flr
INNER JOIN dados_birads brd
    ON flr.ficha = brd.ficha
    AND flr.id_item = brd.id_item
    AND flr.id_subitem = brd.id_subitem
LEFT JOIN refined.saude_preventiva.fleury_retorno_elegivel_ficha eleg
    ON eleg.ficha_origem = flr.ficha
    AND eleg.id_cliente = flr.id_cliente
    AND eleg.linha_cuidado = flr.linha_cuidado
{filtro_ativacao}
"""
 
df_spk = spark.sql(query.format(
    where_clause = where_clause,
    filtro_ativacao = ""
    )
)
 
df_spk_ativacao = spark.sql(query.format(
    where_clause = "",
    filtro_ativacao = filtro_ativacao
    )
)

## Função de Cálculo de Data Prevista de Retorno

Esta célula define a função `calcular_data_prevista`, que adiciona uma coluna com a data recomendada de retorno para exames de acompanhamento com base na classificação BI-RADS. Esta função é crucial para o sistema de notificações, pois determina quando os pacientes devem retornar para exames de controle.

### Detalhes da Função

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark contendo dados dos laudos com a coluna `BIRADS`

**Retorno:**
- DataFrame com a coluna adicional `dth_previsao_retorno`

**Lógica de Cálculo:**
- Para **BI-RADS 1 ou 2**: Adiciona 360 dias (aproximadamente 1 ano) à data do pedido
- Para **BI-RADS 3**: Adiciona 180 dias (aproximadamente 6 meses) à data do pedido
- Para **outros valores de BI-RADS** (4, 5, 6): Define como None (sem recomendação automática)

**Implementação Técnica:**
- Usa `withColumn()` para adicionar uma nova coluna ao DataFrame
- Utiliza `when().otherwise()` para expressões condicionais
- Aplica a função SQL `date_add()` para realizar os cálculos de data

Esta função segue diretrizes médicas padrão para acompanhamento de exames de mamografia baseado na classificação BI-RADS, onde categorias 1-2 representam baixo risco e categoria 3 representa risco intermediário que requer acompanhamento mais próximo.

In [ ]:
def calcular_data_prevista(df_spk: DataFrame):
    """
    Adiciona uma coluna 'dth_previsao_retorno' ao DataFrame com base na coluna 'BIRADS'.

    - Para BIRADS 1 ou 2, adiciona 360 dias à data da coluna 'dth_pedido'.
    - Para BIRADS 3, adiciona 180 dias à data da coluna 'dth_pedido'.
    - Para outros valores de BIRADS, define 'dth_previsao_retorno' como None.

    Parâmetros:
    df_spk (DataFrame): O DataFrame Spark contendo os dados de entrada.

    Retorna:
    DataFrame: O DataFrame atualizado com a nova coluna 'dth_previsao_retorno'.
    """
    df_spk = df_spk.withColumn(
        'dth_previsao_retorno',
        when(
            col('BIRADS').isin([1, 2]),
            expr("date_add(dth_pedido, 360)")
        ).when(
            col('BIRADS') == 3,
            expr("date_add(dth_pedido, 180)")  
        ).otherwise(None)
    )
    return df_spk

## Função de Transformação de Campos

Esta célula define a função `transform_fields`, responsável por enriquecer os DataFrames com campos calculados e transformações adicionais. Esta função é uma peça central do pipeline, pois prepara os dados para análise e persistência após a extração inicial dos valores de BI-RADS.

### Objetivo da Função

A função realiza várias transformações importantes nos dados:
1. Adiciona informação sobre o tempo recomendado de retorno em meses
2. Calcula a data prevista de retorno
3. Converte campos de data para formatos apropriados
4. Calcula a diferença em dias entre datas relevantes

### Parâmetros e Retorno

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark a ser transformado

**Retorno:**
- DataFrame transformado com colunas adicionais

### Lógica Detalhada

1. **Verificação de DataFrame Vazio**:
   - Se o DataFrame estiver vazio, registra um aviso via logger e retorna sem alterações

2. **Adição da Coluna de Retorno**:
   - Cria a coluna `retorno_cliente` que indica o tempo recomendado para retorno em meses
   - BI-RADS 1-2: 12 meses
   - BI-RADS 3: 6 meses
   - Outros valores: 0 (sem recomendação automática)

3. **Cálculo de Datas**:
   - Chama a função `calcular_data_prevista()` definida anteriormente
   - Converte colunas de data para tipo timestamp
   - Calcula a diferença em dias entre a data prevista de retorno e a data do pedido

### Campos Criados/Modificados

- **`retorno_cliente`**: Tempo de retorno recomendado em meses (12, 6 ou 0)
- **`dth_previsao_retorno`**: Data calculada para o próximo exame
- **`dth_pedido_retorno_elegivel`**: Data do pedido convertida para timestamp
- **`dias_ate_retorno`**: Número de dias entre o exame atual e o próximo recomendado

In [ ]:
def transform_fields(df_spk: DataFrame) -> DataFrame:
    """
    Transforma os campos do DataFrame fornecido.

    - Verifica se o DataFrame está vazio. Se estiver, registra um aviso e retorna o DataFrame sem alterações.
    - Adiciona uma coluna 'retorno_cliente' com valores baseados na coluna 'BIRADS':
      - 12 meses para BIRADS 1 ou 2
      - 6 meses para BIRADS 3
      - 0 para outros valores
    - Calcula a data prevista de retorno usando a função 'calcular_data_prevista'.
    - Converte a coluna 'dth_pedido_retorno_elegivel' para o tipo timestamp.
    - Converte a coluna 'dth_previsao_retorno' para o tipo timestamp.
    - Calcula a diferença em dias entre 'dth_previsao_retorno' e 'dth_pedido', armazenando o resultado na coluna 'dias_ate_retorno'.

    Parâmetros:
    df_spk (DataFrame): O DataFrame a ser transformado.

    Retorna:
    DataFrame: O DataFrame transformado com as novas colunas.
    """
    if df_spk.isEmpty():
        logger.warning("No Data Found!")
        return df_spk
    
    df_spk = df_spk.withColumn(
        'retorno_cliente',
        when(col('BIRADS').isin([1, 2]), 12).when(col('BIRADS') == 3, 6).otherwise(0)
    )

    df_spk = calcular_data_prevista(df_spk)
    df_spk = df_spk.withColumn('dth_pedido_retorno_elegivel', to_timestamp(col('dth_pedido_retorno_elegivel')))
    df_spk = df_spk.withColumn('dth_previsao_retorno', to_timestamp(col('dth_previsao_retorno')))
    df_spk = df_spk.withColumn('dias_ate_retorno', datediff(to_date(col('dth_previsao_retorno')), to_date(col('dth_pedido'))))
    return df_spk

## Configuração de Webhook e Função de Tratamento de Erros

Esta célula configura o sistema de alerta e define a função `error_message` para tratamento padronizado de erros. O tratamento de erros é fundamental para garantir a robustez do pipeline e facilitar o diagnóstico de problemas.

### Variáveis e Configurações

- **`WEBHOOK_DS_AI_BUSINESS_STG`**: Define o ambiente para o webhook do Sentinel ('stg' para staging/homologação)

### Detalhes da Função `error_message`

A função `error_message` é responsável por tratar e reportar erros de maneira padronizada:

**Parâmetros:**
- `e` (Exception): A exceção capturada durante a execução

**Comportamento:**
1. **Captura detalhada do erro**: Utiliza `traceback.format_exc()` para obter a pilha de chamadas completa
2. **Formatação da mensagem**: Cria uma mensagem informativa com detalhes do erro
3. **Envio de alerta**: Inicializa o Sentinel com parâmetros específicos e envia o alerta:
   - `project_name`: 'Monitor_Linhas_Cuidado_Mama'
   - `task_title`: 'Fleury Mamografia'
   - Categoria do alerta: 'Alerta'
4. **Exibição do traceback**: Exibe o traceback no console para depuração imediata
5. **Propagação do erro**: Relança a exceção para interromper a execução do pipeline

Esta função centraliza o tratamento de erros em todo o notebook, garantindo consistência no monitoramento e notificação de problemas.

In [ ]:
WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

def error_message(e):
    """
    Envia alerta para o Sentinel e exibe o traceback em caso de erro ao salvar dados.

    Parâmetros:
        e (Exception): Exceção capturada.

    Comportamento:
        - Formata o traceback do erro.
        - Envia alerta para o Sentinel com detalhes do erro.
        - Exibe o traceback no console.
        - Relança a exceção.
    """
    error_message = traceback.format_exc()
    summary_message = f"""Erro ao salvar dados.\n{error_message}"""
    sentinela_ds_ai_business = Sentinel(
        project_name='Monitor_Linhas_Cuidado_Mama',
        env_type=WEBHOOK_DS_AI_BUSINESS_STG,
        task_title='Fleury Mamografia'
    )
    sentinela_ds_ai_business.alerta_sentinela(
        categoria='Alerta', 
        mensagem=summary_message,
        job_id_descritivo='1_fleury_mama_birads'
    )
    traceback.print_exc()
    raise e

## Funções de Gerenciamento de Dados em Delta Lake

Esta célula define três funções essenciais para gerenciar a persistência de dados no formato Delta Lake:

### 1. Função `insert_data`

**Objetivo:** Inserir dados em uma tabela Delta, sobrescrevendo completamente o conteúdo anterior.

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem inseridos
- `table_name` (str): Nome da tabela Delta de destino
- `serializable` (bool, opcional): Define o nível de isolamento da transação

**Comportamento:**
- Registra a operação no log
- Configura o nível de isolamento apropriado ('Serializable' ou 'WriteSerializable')
- Escreve os dados com opções otimizadas:
  - Permite atualização do schema se necessário (`overwriteSchema=true`)
  - Usa modo 'overwrite' para substituição completa dos dados
- Captura exceções e aciona o tratamento de erros via `error_message()`

### 2. Função `merge_data`

**Objetivo:** Realizar operações de mesclagem (upsert) em tabelas Delta existentes.

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem mesclados
- `table_name` (str): Nome da tabela Delta de destino

**Comportamento:**
- Registra a operação no log
- Cria uma view temporária (`increment_birads`) com os dados do DataFrame
- Executa um comando MERGE SQL que:
  - Atualiza registros existentes (identificados pela chave composta `ficha`, `id_item`, `id_subitem`)
  - Insere novos registros quando não existem correspondências
- Captura exceções e aciona o tratamento de erros via `error_message()`

### 3. Função `save_data`

**Objetivo:** Função de alto nível que decide entre inserção ou mesclagem com base no contexto.

**Parâmetros:**
- `df_spk` (DataFrame): DataFrame Spark com os dados a serem salvos
- `table_name` (str): Nome da tabela Delta de destino
- `serializable` (bool, opcional): Parâmetro passado para `insert_data` se usado

**Lógica:**
- Verifica se o DataFrame está vazio - se estiver, retorna sem ação
- Verifica se a tabela já existe:
  - Se existir, chama `merge_data()` para realizar upsert
  - Se não existir, chama `insert_data()` para criar a tabela

Este conjunto de funções fornece uma abstração completa para persistência de dados, gerenciando automaticamente a criação de novas tabelas ou atualização das existentes, com tratamento adequado de erros.

In [ ]:
def insert_data(df_spk: DataFrame, table_name: str, serializable: bool = False):
    """
    Insere os dados do DataFrame em uma tabela Delta no Databricks.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser salvo.
        table_name (str): Nome da tabela destino.
        serializable (bool): Define o nível de isolamento da transação Delta. 
            Se True, usa 'Serializable', caso contrário 'WriteSerializable'.
    
    Comportamento:
        - Escreve o DataFrame na tabela Delta especificada, sobrescrevendo dados existentes.
        - Atualiza o schema da tabela conforme necessário.
        - Define o nível de isolamento da transação Delta.
    """
    try:
        logger.info(f"Inserting Data: {table_name}")
        isolation_level = 'Serializable' if serializable else 'WriteSerializable'
        (
            df_spk.write
                .format('delta')
                .option('overwriteSchema', 'true')
                .option('delta.isolationLevel', isolation_level)
                .mode('overwrite')
                .saveAsTable(table_name)
        )
    except Exception as e:
        error_message(e)
 
def merge_data(df_spk: DataFrame, table_name: str):
    """
    Realiza um merge (upsert) dos dados do DataFrame em uma tabela Delta existente.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser mesclado.
        table_name (str): Nome da tabela destino.
    
    Comportamento:
        - Cria uma view temporária com os dados do DataFrame.
        - Executa um comando MERGE SQL para atualizar registros existentes (com base em ficha, id_item e id_subitem)
          ou inserir novos registros na tabela Delta.
    """
    try:
        logger.info(f"Merging Data: {table_name}")
        df_spk.createOrReplaceTempView(f"increment_birads")
        merge_query = f"""
            MERGE INTO {table_name} AS target
            USING increment_birads AS source
                ON target.ficha = source.ficha
                AND target.id_item = source.id_item
                AND target.id_subitem = source.id_subitem
            WHEN MATCHED THEN
                UPDATE SET *
            WHEN NOT MATCHED THEN
                INSERT *
        """
        spark.sql(merge_query)
    except Exception as e:
        error_message(e)
 
def save_data(df_spk: DataFrame, table_name: str, serializable: bool = False):
    """
    Salva os dados do DataFrame em uma tabela Delta, realizando merge se a tabela já existir.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame Spark a ser salvo.
        table_name (str): Nome da tabela destino.
        serializable (bool): Define o nível de isolamento da transação Delta na inserção inicial.
    
    Comportamento:
        - Se o DataFrame estiver vazio, não faz nada.
        - Se a tabela já existe, realiza merge (upsert) dos dados.
        - Se a tabela não existe, insere os dados criando a tabela.
    """
    if df_spk.isEmpty():
        return None
 
    if spark.catalog.tableExists(table_name):
        merge_data(df_spk, table_name)
    else:
        insert_data(df_spk, table_name, serializable)

## Aplicação das Transformações nos DataFrames

Esta célula aplica as transformações definidas anteriormente aos dois DataFrames principais gerados pela consulta SQL. Este é um passo essencial para preparar os dados para análise e persistência.

### Objetivo da Célula

Enriquecer os DataFrames `df_spk` e `df_spk_ativacao` com:
- Tempo recomendado para retorno (em meses)
- Datas previstas para retorno
- Conversões adequadas de tipos de dados
- Cálculo de dias até o retorno previsto

### Dependências

- Função `transform_fields()` definida anteriormente
- DataFrames `df_spk` e `df_spk_ativacao` criados pela consulta SQL

### Lógica Detalhada

A célula simplesmente chama a função `transform_fields()` para cada DataFrame:

1. Para `df_spk` (todos os laudos com classificação BI-RADS)
2. Para `df_spk_ativacao` (subset de pacientes elegíveis para notificações)

### Impacto

Após a execução desta célula:
- Os DataFrames terão novas colunas como `retorno_cliente`, `dth_previsao_retorno` e `dias_ate_retorno`
- As datas estarão convertidas para o formato apropriado (timestamp)
- Os DataFrames estarão prontos para análise e para serem salvos nas tabelas Delta

In [ ]:
# Aplicar transformações nos dfs
df_spk = transform_fields(df_spk)
df_spk_ativacao = transform_fields(df_spk_ativacao)

## Deduplicação de Registros para Ativação

Esta célula realiza um processamento importante no DataFrame de ativação, removendo registros duplicados para garantir que cada paciente receba apenas uma notificação por ficha, independentemente de quantos exames (itens) foram realizados.

### Objetivo da Célula

Eliminar registros duplicados do DataFrame `df_spk_ativacao` para que cada ficha (que representa um atendimento) apareça apenas uma vez na lista de ativação, evitando o envio de múltiplas notificações para o mesmo paciente sobre o mesmo exame.

### Lógica Implementada

A célula utiliza o método `.dropDuplicates(['ficha'])` do PySpark DataFrame API para:

1. Identificar registros com o mesmo valor na coluna `ficha`
2. Manter apenas a primeira ocorrência de cada valor único de `ficha`
3. Descartar todas as ocorrências duplicadas

### Contexto e Importância

Na estrutura de dados do Fleury:
- Uma `ficha` corresponde a um atendimento/visita
- Um atendimento pode conter vários `id_item` e `id_subitem` (exames e procedimentos)

Sem esta deduplicação, o sistema enviaria múltiplas notificações para o mesmo paciente sobre o mesmo atendimento, apenas porque foram realizados vários procedimentos durante aquela visita. A deduplicação garante que a comunicação com o paciente seja apropriada e não excessiva.

In [ ]:
# Excluir duplicados para considerar apenas a ficha na ativação e não os exames (itens). Assim vamos enviar apenas 
# 1 push por ficha
df_spk_ativacao = df_spk_ativacao.dropDuplicates(['ficha'])

## Contagem de Laudos Processados

Esta célula exibe a quantidade de registros em cada DataFrame após todas as transformações e filtragens. Este passo fornece um importante controle de qualidade e visibilidade sobre o volume de dados que serão persistidos.

### Objetivo da Célula

- Contar e exibir o número de registros nos DataFrames principais
- Fornecer uma verificação visual do volume de dados processados
- Permitir validar se os filtros e transformações estão gerando a quantidade esperada de registros

### Dados Exibidos

Duas métricas principais são exibidas:

1. **Quantidade de laudos na tabela principal**: Número total de laudos de mamografia com classificação BI-RADS extraída e processada
   
2. **Quantidade de laudos na tabela de ativação**: Número de registros únicos (por ficha) que atendem aos critérios para notificação:
   - BI-RADS 1, 2 ou 3
   - Pacientes do sexo feminino entre 40-75 anos
   - Sem retorno já registrado
   - De marcas específicas

### Importância para o Pipeline

Estas contagens servem tanto para fins de monitoramento quanto para diagnóstico:
- Valores muito baixos podem indicar problemas nos filtros ou na extração de BI-RADS
- Valores muito altos podem sugerir processamento duplicado ou filtros muito amplos
- A comparação entre as duas contagens dá uma visão da proporção de pacientes elegíveis para notificação

In [ ]:
print('quantidade de laudos salvos na tabela',df_spk.count())
print('quantidade de laudos salvos na tabela de ativação', df_spk_ativacao.count())

## Salvamento das Tabelas Processadas

Esta célula final realiza a persistência dos dados processados nas tabelas Delta Lake, concluindo o pipeline de processamento. Os dados serão utilizados para análises posteriores e para o sistema de notificação de pacientes.

### Objetivo da Célula

Salvar os dois DataFrames processados nas suas respectivas tabelas Delta:

1. **Tabela Principal (BI-RADS)**: Contém todos os laudos com classificação BI-RADS extraída e dados enriquecidos
2. **Tabela de Ativação**: Contém apenas os registros elegíveis para envio de notificações

### Funções Utilizadas

- **`save_data()`**: Para a tabela principal, utiliza a função inteligente que decide entre merge e insert com base na existência da tabela
- **`insert_data()`**: Para a tabela de ativação, força a substituição completa dos dados, garantindo que apenas os pacientes atualmente elegíveis estejam presentes

### Comportamento de Persistência

- Para tabela principal: Dados são mesclados (merge) se a tabela já existe, preservando o histórico
- Para tabela de ativação: Dados são completamente substituídos, mantendo apenas os pacientes atualmente elegíveis

### Tabelas de Destino

- **`refined.saude_preventiva.fleury_laudos_mama_birads`**: Tabela histórica com todos os laudos processados
- **`refined.saude_preventiva.fleury_laudos_mama_birads_ativacao`**: Tabela com pacientes atualmente elegíveis para notificação

Após a execução desta célula, os dados estarão disponíveis para consumo por outros sistemas, como dashboards de BI ou sistemas de notificação automática de pacientes.

In [ ]:
# # Salvar tabelas
save_data(df_spk, table_birads)
insert_data(df_spk_ativacao, table_birads_ativacao)